## Numerical Methods
### Activity 2: General Linear Regression

First load the data into a pandas dataframe and display it. Our data is from a scientific paper from 1995 comparing levels of various chemicals to the age of carpets.

In [ ]:
%matplotlib notebook
import pandas as pd
df = pd.read_csv('carpet_age.csv')
display(df)
df.describe()

Now we'll extract the data into numpy arrays for regression. We'll also do some light data cleaning to remove data points with missing values.

In [ ]:
import numpy as np

# We have to extract the data from the columns we're interested in.
# X will collect the independent variables, and y the dependent (response) variable.

X_data = np.array(df[['cys_acid', 'cys', 'met', 'tyr']].values)
y_data = np.array(df[['age']].values.flatten())

# Notice that the bottom two entries don't have ages. To keep things simple we'll remove them.
# Numpy's delete operation doesn't actually modify the arrays it operates it, it just returns a new array.
# So we create two new arrays to do our regression with.
y = np.delete(y_data,[len(y_data)-1, len(y_data)-2],0)
X = np.delete(X_data,[len(X_data)-1, len(X_data)-2],0)

print(X)
print(y)



Now we can calculate the parameters of the regression hyperplane $a_0 + a_1x_1 + a_2x_2 + a_3x_3 + a_4x_4$. This is done using scikit-learn's LinearRegression class, just like for simple linear regression. 

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# We run the 'fit' function on X and y to get our regression model.
model.fit(X,y)

# intercept_ returns a_0, and coef_ returns [a_1,a_2,a_3,a_4], so we get out array of parameters by combining these
A = np.append(model.intercept_, model.coef_) 

print('parameters:', A)



Again just like with simple linear regression, we can easily find the coefficient of determination $r^2$ here too.

In [ ]:
r2 = round(model.score(X, y),3)
print('coefficient of determination:', r2)

Write a function that takes as input the array of data for the independent variables ($X$), and the array of data for the 
dependent vaiable ($y$), and also the array of parameters calculated for the multiple regression model (for us this is
$A = [a_0,a_1,a_2,a_3,a_4]$), and outputs the correct value for the standard error of the estimate $s_{yx}$ (I think it should 
be 46.379).

In [ ]:
import math

def findSE(x_values, y_values, A):
    #TODO
    return 

s_yx = findSE(X,y,A)
print('standard error of estimate:', s_yx)

The regression we've just done involves 4 independent variables, so the regression hypersurface is a 4 dimensional shape in 5 dimensional space. This is hard to draw. To get a nice picture we'll restrict to just the first two independent vatriiables.

In [ ]:
X_2 = np.array(df[['cys_acid', 'cys']].values)[:-2]

print(X_2)

Now we can proceed like before.

In [ ]:
model = LinearRegression()
model.fit(X_2,y)

# Here A_2 will be [a_0,a_1,a_2]
A_2 = np.append(model.intercept_, model.coef_) 

print('parameters: {}'.format(A_2))

r2_new = round(model.score(X_2, y),3)
print('coefficient of determination: {}'.format(r2_new))

We can also draw a picture. You should be able to drag the image to change the view. 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator

x1_values = np.delete(X_2,1,1)
x2_values = np.delete(X_2,0,1)
z_values = y
x_line = np.linspace(min(x1_values), max(x1_values) + 1, 100)
y_line = np.linspace(min(x2_values), max(x2_values) + 1, 100)
X, Y = np.meshgrid(x_line, y_line)
Z = A_2[0] + X*A_2[1] + Y*A_2[2]
colours =[]
for i in range(len(x1_values)):
    if z_values[i] > A_2[0] + x1_values[i]*A_2[1] + x2_values[i]*A_2[2]:
        colours.append('red')
    else:
        colours.append('blue')
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.w_zaxis.set_major_locator(LinearLocator(6))
ax.scatter(x1_values, x2_values, z_values, marker = 'o', c = colours)
ax.plot_surface(X, Y, Z, cmap=cm.viridis, linewidth=0, alpha = 0.5)
ax.view_init(elev=10., azim=25)
plt.show()




Now let's write our own function for performing multiple regression with 2 independent $x$ variables. This function will take an 'array of arrays' representing the values of 2 independent $x$ variables, and an array of values of $y$, and it will output the parameters $a_0,a_1,a_2$ of the regression plane. We will need to use numpy's linear algebra capabilities to solve the matrix equation that gives us the solution.

In other words, our function will perform multiple regression as it is done in the example form the class. This is just a practice example. In reality we would not implement this function like this, as it is limited to just 2 indepenedent variables. For more flexibility we would implement the general method using the formula $A = (Z^TZ)^{-1}Z^TY$ (see the class material, and also later in this notebook), or depending on the data we have it could be faster to solve the optimization problem numerically using something like gradient descent.  

In [ ]:
def mRegression(x_values, y_values):
    # We need to populate the matrix from the notes with sums of x values.
    x_values = np.array(x_values)
    y_values = np.array(y_values)
    n = len(y_values)
    Sx_1 = np.sum(x_values,0)[0]
    Sx_2 = np.sum(x_values,0)[1]
    Sx_1x_2 = np.sum(x_values[:,0]*x_values[:,1])
    Sx_1_square = np.sum(x_values[:,0]**2)
    Sx_2_square = np.sum(x_values[:,1]**2)
    Sy = np.sum(y_values)
    Sx_1y = np.sum(x_values[:,0]*y_values)
    Sx_2y = np.sum(x_values[:,1]*y_values)
    M = np.array([[n, Sx_1, Sx_2], [Sx_1, Sx_1_square, Sx_1x_2], [Sx_2, Sx_1x_2, Sx_2_square]])
    col = [Sy, Sx_1y, Sx_2y]    
    return np.linalg.solve(M, col)

# We can check this gives the same answer as we got using model.fit earlier.
print(mRegression(X_2,y))

Now we will tackle a general regression problem. 

Suppose the number of particles in a system decays exponentially according to the model 

$$p(t) = ae^{-1.5t} + be^{-0.3t}+ce^{-0.05t}.$$

Here $p(t)$ is the number of particles remaining at time $t$. 

We want to find the values $a,b,c$ so we fully understand this model. To do this we run an experiment and collect data at various times. This is given in the following table:

|  t  |  p(t)  |
|-----|--------|
| 0.5 | 6      |
| 1   | 4.4    |
| 2   | 3.2    |
| 3   | 2.7    |
| 4   | 2      |
| 5   | 1.9    |
| 6   | 1.7    |
| 7   | 1.4    |
| 9   | 1.1    |



To solve this we will use the general least squares regression method. Here $y = az_0 + bz_1 + cz_2$, where $z_0, z_1$, and $z_2$ are 'basis functions' of $t$.

In this case $z_0 = e^{-1.5t}$, $z_1 = e^{-0.3t}$ and $z_2 = e^{-0.05t}$.

Using these basis functions and the values of $t$ given in the data we create a matrix $Z$.
We also get a vector $Y$ from the values of $p(t)$.
We find the values of $a,b,c$ by evaluating the matrix formula $(Z^TZ)^{-1}Z^TY$.

In [ ]:
from math import exp

t_values = [0.5, 1, 2, 3, 4, 5, 6, 7, 9]
Y = np.array([[6], [4.4], [3.2], [2.7], [2], [1.9], [1.7], [1.4], [1.1]])

# Returns the matrix Z as an array. 
# You need to write this part.
# You can find information about Z in the notes.
def make_Z(t = t_values):
    # TODO
    return np.array(A)


# Now using the matrix Z we can evaluate a matrix formula to find the parameters a,b,c.
Z = make_Z()
Zt = np.transpose(Z)
A = np.matmul(np.matmul(np.linalg.inv(np.matmul(Zt,Z)),Zt),Y)
print(A)


# If my calculation is correct, this should produce 
# [[4.13749658]
#  [2.89588175]
#  [1.53491995]]
# This corresponds to a = 4.137, b = 2.896, c = 1.535 (to 3 decimal places). 
# This gives us an estimate for the parameters of the model.



You can test how good this model is by comparing the values predicted if you use these values for $a,b,c$ with the model for $p(t)$ at the $t$ values given in the data with the observed values of $p(t)$. E.g. you can calcuate $S_t$ and $S_r$, and so also $r^2$ and $s_{yx}$. We will just draw a picture.

In [ ]:
fig, ax = plt.subplots() 
ax.scatter(t_values, Y.flatten(), c = 'orange')
A = np.array([[4.13749658],  [2.89588175], [1.53491995]])
a,b,c = A.flatten()
x_line = np.linspace(0,10,100)
y_line = a*np.exp(-1.5*x_line) + b*np.exp(-0.3*x_line) + c*np.exp(-0.05*x_line) 
# we needed the numpy exp function in the line above to draw the line properly
plt.plot(x_line, y_line)
plt.show()


